### Chapter 7: Synchronization Examples

**Review Question 7.1**
*   **Question:** What are the names of the two processes associated with the bounded-buffer problem?
*   **Answer:** Producer and Consumer.

**Review Question 7.2**
*   **Question:** How many writers may concurrently share the database with the readers-writers problem?
*   **Answer:** Only one writer may have exclusive access to the database at a time.

**Review Question 7.3**
*   **Question:** What is the problem if all philosophers simultaneously pick up their left fork?
*   **Answer:** Deadlock. Each philosopher would hold one fork and wait indefinitely for their right fork (which is held by their neighbor), creating a circular wait.

**Review Question 7.4**
*   **Question:** What are the two states of a Windows dispatcher object?
*   **Answer:** Signaled state and nonsignaled state.

**Review Question 7.5**
*   **Question:** What is available in Linux for updating an integer variable without having to use locks?
*   **Answer:** Atomic integers (the `atomic_t` type and associated atomic operations like `atomic_inc()`, `atomic_dec()`, `atomic_read()`, etc.).

**Review Question 7.6**
*   **Question:** True or False? Linux uses spinlocks for both single and multiple processor systems.
*   **Answer:** False. Linux disables kernel preemption for spinlocks on single-processor systems (making them act as no-ops in non-preemptive sections) and uses actual busy-waiting spinlocks only on multiprocessor systems.

**Review Question 7.7**
*   **Question:** What are the Pthreads operations for locking and unlocking a mutex lock?
*   **Answer:** `pthread_mutex_lock()` and `pthread_mutex_unlock()`.

**Review Question 7.8**
*   **Question:** Provide at least one alternative to mutex locks, semaphores, reader-writer locks, and monitors that provide support for concurrent programming.
*   **Answer:** OpenMP (using compiler directives for parallelism), functional programming languages (which use immutable data and avoid state), transactional memory, or lock-free/wait-free data structures (using atomic operations like CAS).